# Report 3

## Introduction

This week's report was focused on generating choropleth maps, that is, plots that use shading, color, or other properties to indicate values with respect to some metrics in different geographical areas. An example of a choropleth map commonly seen could be an election map with red and blue shades indicating votes across the country, as seen below (source: https://upload.wikimedia.org/wikipedia/commons/4/49/2016_Nationwide_US_presidential_county_map_shaded_by_vote_share.svg) <img src='https://upload.wikimedia.org/wikipedia/commons/4/49/2016_Nationwide_US_presidential_county_map_shaded_by_vote_share.svg'>

However, the point of this exercise was for us to generate our own. The first thing I needed to do was to figure out exactly what data I wanted to represent.

## Exploring Data

After googling around for datasets to use, I eventually stumbled upon the government census website: https://www.census.gov/support/USACdataDownloads.html.

It seemed unlikely that I could find a more thorough and well organized source of data anywhere else, so I began exploring what was available. The data was split into different categories like health, income and agriculture. Eventaully I decided to specifically focus on the data associated with age.

I downloaded the folder containing the age-specific data and found 4 excel files. Each file contained a number of columns with each row corresponding to a county in the U.S. as shown here:

<img src="data_screenshot.png"/>

Columns such as "Areaname" were straighfoward, and "STCOU" was evidently enough a FIPS code for each area which would be import later on. However, I needed to do more digging to find out what the other column names such as "AGE050180D" meant.

I eventually found a file called "Mastdata" which described every single column name from every possible data collection. Upon inspection it looked like so:

<img src="mastscreenshot.png"/>

I just needed the first two columns which had the code and corresponding description to find out what the columns in my other file meant. With these two files I was ready to get to work.

## Loading Data

I used the pandas library in Python to do all of the data processing. The first step was to save these excel files as CSV (comma seperated values) files so that I could load them into a pandas dataframe. I also imported `math` and `etree` for later use:

In [3]:
import pandas
import math
from lxml import etree

df = pandas.read_csv("AGE012.csv")
mast = pandas.read_csv('mastdata.csv')

The resulting dataframe for the age data resembles what appears in the spreadsheet:

<img src="df.png"/>

And likewise for the descriptions:

<img src="mastdf.png"/>

## Interpretation

Now, my next step was to figure out, using the descriptions, what the data in my csv file was actually describing. I used a `dict` mapping each coded column name to a description and populated it with the data from "Mastdata", where the first column was the code and the second column was the corresponding description. I named the `dict` "codes_to_names":

In [4]:
codes_to_names = {}
for item in mast.iterrows():
    codes_to_names[item[1][0]] = item[1][1]

I could then use this `dict` to go through and look up the descriptions of all my columns. Some of them didn't seem to have an available description, but plenty did:

In [5]:
for col in df.columns:
    print(col, codes_to_names.get(col))

Areaname None
STCOU None
AGE040205F None
AGE040205D Resident population total (July 1 - estimate) 2005
AGE040205N1 None
AGE040205N2 None
AGE040206F None
AGE040206D Resident population total (July 1 - estimate) 2006
AGE040206N1 None
AGE040206N2 None
AGE040207F None
AGE040207D Resident population total (July 1 - estimate) 2007
AGE040207N1 None
AGE040207N2 None
AGE040208F None
AGE040208D Resident population total (July 1 - estimate) 2008
AGE040208N1 None
AGE040208N2 None
AGE040209F None
AGE040209D Resident population total (July 1 - estimate) 2009
AGE040209N1 None
AGE040209N2 None
AGE050180F None
AGE050180D Resident population: Median age (complete count) 1980
AGE050180N1 None
AGE050180N2 None
AGE050190F None
AGE050190D Resident population: Median age (complete count) 1990
AGE050190N1 None
AGE050190N2 None
AGE050200F None
AGE050200D Resident population: Median age (April 1 - complete count) 2000
AGE050200N1 None
AGE050200N2 None
AGE050210F None
AGE050210D Resident population: Median age (

I noticed that there were columns giving the median age for each county for the years 1980, 1990, 2000, and 2010. I decided to work with those columns and see what kind of results I would get. I first manually put those codes into my own `dict` so I could use them later:

In [6]:
median_age_code = {
    '1980': 'AGE050180D',
    '1990': 'AGE050190D',
    '2000': 'AGE050200D',
    '2010': 'AGE050210D'
}

## Processing

Since I had the in these columns the corresponding median age for every county in the US for a respective year, I wanted to be able store this more conveniently in a `dict` so that I could do a simple lookup by the county code. I wrote a function to do this which takes as input a year and returns the corresponding dict for that year (making use of `median_age_code`). One small adjustment I made was prepending 0's to the FIPS codes to ensure they were all 5 digits, as leading 0's seemed to have been lost in the CSV file. I would need this uniformity later on when working with my map. I also kept track of the max and min medians for all counties to keep in mind as bounds when visually representing the data:

In [10]:
def load_median_ages(year):    
    median_ages = {}
    code = median_age_code[year]
    minv = maxv = 30
    for item in df.iterrows():
        fips = str(item[1]['STCOU'])
        if len(fips) < 5:
            fips = '0'*(5-len(fips)) + fips
        median_ages[fips] = item[1][code]
        if item[1][code] < minv:
            minv = item[1][code]
        if item[1][code] > maxv:
            maxv = item[1][code]
    #print(minv,maxv)
    return median_ages

## Visualizing with SVG
Now that I had all the counties and their corresponding median ages stored in a simple `dict`, I just needed to visually represent the data. This could seem like a daunting task to someone like myself with little experience in design or graphics, but it is actually pretty straightfoward with the right building blocks.

SVG, which stands for scalable vector graphics, is a markup language that makes it easy to programatically draw graphics that can be rendered in the browser. SVG can be created/modified with code and also exported from a graphics program such as Inkscape.

To visualize my data, I of course needed a map of the United States which could easily be found online in SVG format:

<img src="USA_Counties_with_FIPS_and_names.svg"/>

Inspecting the code for this SVG file, it basically consists of a long list of `path` elements for each county:

<img src="svg_code.png"/>

That might look like a lot, but there are only two small things we care about: the `style` and the `id` attributes. Upon inspection, the `id` attribute actually corresponds to the FIPS code for the county. This gives us a way of mapping our data in the Python program to a node in the SVG, excellent! The style attribute contains a setting called "fill" which is set to a default of "#d0d0d0", the hex color code that gives every county a grey backgound.

All else that is needed to represent the data is look up the median age based on each FIPS code with our `dict` and tweak the corresponding "fill" color to represent it somehow. After modifying the SVG in this way, the resulting image will be a legitimate choropleth map instead of the current all-grey one.

## Modifying the Map

I decided to make green represent younger populations and red represent older populations, leaving grey in the middle to represent the overall median age (which between all 4 years ended up being about 34 years).

I wrote a function to load the plain SVG (with an added color key on the side according to my above description), iterate through all the nodes for each county, and modify them to be the appropriate color based on their median ages. I also needed to save the map to a new file in a separate function called `write_map`:

In [13]:
overall_avg = 33.85
def gen_map(median_ages):
    with open('drawing.svg') as f:
        doc = etree.fromstring(f.read().encode('utf-8'))
    for item in doc[3]:
        if 'style' in item.attrib and 'fill:#d0d0d0' in item.attrib['style'] and item.attrib['id'] in median_ages:
            avg_age = median_ages[item.attrib['id']]
            r = 'd0'
            g = 'd0'
            b = 'd0'
            if avg_age - overall_avg > 1:
                r = 'f0'
                magnitude = math.ceil(min(6,0.5*(avg_age - overall_avg)))
                shadow = str(hex(13 - magnitude))[2:]
                g = shadow + g[1:]
                b = shadow + b[1:]
            elif overall_avg - avg_age > 1:
                g = 'f0'
                magnitude = math.ceil(min(13,0.5*(overall_avg - avg_age)))
                shadow = str(hex(13 - magnitude))[2:]
                r = shadow + r[1:]
                b = shadow + b[1:]
            item.attrib['style'] = item.attrib['style'].replace('fill:#d0d0d0', 'fill:#'+r+g+b)
    return etree.tostring(doc)

def write_map(year, age_map):
    with open('median_' + year + '.svg','w') as f:
        #print("Writing", f.name)
        f.write(age_map.decode('utf-8'))

Now we can use our functions to generate the maps for each year of data:

In [14]:
for year in median_age_code:
    median_ages = load_median_ages(year)
    age_map = gen_map(median_ages)
    write_map(year, age_map)

## Results

### Median Age (Years) - 1980

<img src="1980_ss.png"/>

### Median Age (Years) - 1990

<img src="1990_ss.png"/>

### Median Age (Years) - 2000

<img src="2000_ss.png"/>

### Median Age (Years) - 2010

<img src="2010_ss.png"/>

## Conclusion

There is a pretty obvious trend here: America is getting older, with few exceptions.

It would be interesting to compare this to the rest of the World, and maybe juxtapose this with other health or income related data sets to see any possible correlations. For the sake of this report I will leave it at this though.

## Code

In [2]:
import pandas
import math
from lxml import etree

df = pandas.read_csv("AGE012.csv")
mast = pandas.read_csv('mastdata.csv')

codes_to_names = {}
for item in mast.iterrows():
    codes_to_names[item[1][0]] = item[1][1]
    
for col in df.columns:
    print(col, codes_to_names.get(col))
    
median_age_code = {
    '1980': 'AGE050180D',
    '1990': 'AGE050190D',
    '2000': 'AGE050200D',
    '2010': 'AGE050210D'
}

def load_median_ages(year):    
    median_ages = {}
    code = median_age_code[year]
    i = 0
    minv = maxv = 30
    for item in df.iterrows():
        fips = str(item[1]['STCOU'])
        if len(fips) < 5:
            fips = '0'*(5-len(fips)) + fips
        median_ages[fips] = item[1][code]
        if item[1][code] < minv:
            minv = item[1][code]
        if item[1][code] > maxv:
            maxv = item[1][code]
    print(minv,maxv)
    return median_ages

overall_avg = 33.85
def gen_map(median_ages):
    with open('drawing.svg') as f:
        doc = etree.fromstring(f.read().encode('utf-8'))
    for item in doc[3]:
        if 'style' in item.attrib and 'fill:#d0d0d0' in item.attrib['style'] and item.attrib['id'] in median_ages:
            avg_age = median_ages[item.attrib['id']]
            r = 'd0'
            g = 'd0'
            b = 'd0'
            if avg_age - overall_avg > 1:
                r = 'f0'
                magnitude = math.ceil(min(6,0.5*(avg_age - overall_avg)))
                shadow = str(hex(13 - magnitude))[2:]
                g = shadow + g[1:]
                b = shadow + b[1:]
            elif overall_avg - avg_age > 1:
                g = 'f0'
                magnitude = math.ceil(min(13,0.5*(overall_avg - avg_age)))
                shadow = str(hex(13 - magnitude))[2:]
                r = shadow + r[1:]
                b = shadow + b[1:]
            item.attrib['style'] = item.attrib['style'].replace('fill:#d0d0d0', 'fill:#'+r+g+b)
    return etree.tostring(doc)

def write_map(year, age_map):
    with open('median_' + year + '.svg','w') as f:
        print("Writing", f.name)
        f.write(age_map.decode('utf-8'))
    
for year in median_age_code:
    median_ages = load_median_ages(year)
    age_map = gen_map(median_ages)
    write_map(year, age_map)

Areaname None
STCOU None
AGE040205F None
AGE040205D Resident population total (July 1 - estimate) 2005
AGE040205N1 None
AGE040205N2 None
AGE040206F None
AGE040206D Resident population total (July 1 - estimate) 2006
AGE040206N1 None
AGE040206N2 None
AGE040207F None
AGE040207D Resident population total (July 1 - estimate) 2007
AGE040207N1 None
AGE040207N2 None
AGE040208F None
AGE040208D Resident population total (July 1 - estimate) 2008
AGE040208N1 None
AGE040208N2 None
AGE040209F None
AGE040209D Resident population total (July 1 - estimate) 2009
AGE040209N1 None
AGE040209N2 None
AGE050180F None
AGE050180D Resident population: Median age (complete count) 1980
AGE050180N1 None
AGE050180N2 None
AGE050190F None
AGE050190D Resident population: Median age (complete count) 1990
AGE050190N1 None
AGE050190N2 None
AGE050200F None
AGE050200D Resident population: Median age (April 1 - complete count) 2000
AGE050200N1 None
AGE050200N2 None
AGE050210F None
AGE050210D Resident population: Median age (